In [107]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import re
from numpy import NaN

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
    }

dta = pd.read_csv('letterboxd\watched.csv')[['Name','Letterboxd URI']]


pttrn = [r'[0-9][.][0-9][0-9]', r'>[A-Za-z]+<', r'[0-9]+', r'>[A-Za-z,()]+<']


data = dict()
user = 'gabrielcooutin'
r = requests.get("https://letterboxd.com/"+user+"/films/page/1", headers=header)
site = BeautifulSoup(r.text,'html.parser')
string = str(site.findAll('li', {'class':'paginate-page'})[-1])
address = "https://letterboxd.com"


# Regex para o nome e o link
pttrn = r'>([0-9])<'
pttrn2 = r'img alt="(.*?)"'
pttrn3 = r'data-target-link="(.*?)"'

num = int(re.findall(pttrn,string)[0])

# Listas para armazenar os dados
names = []
links = []

for i in range(1,num+1):
  r = requests.get("https://letterboxd.com/"+user+f"/films/page/{i}", headers=header)
  site = BeautifulSoup(r.text,'html.parser')

  # Iterando pelos elementos de interesse
  for i in site.findAll('li', {"class": "poster-container"}):
      name_match = re.search(pttrn2, str(i))
      link_match = re.search(pttrn3, str(i))
      
      if name_match and link_match:
          names.append(name_match.group(1))  # Primeiro grupo do nome
          links.append(link_match.group(1))  # Primeiro grupo do link

# Criando o DataFrame
df = pd.DataFrame({
    "Name": names,
    "Link": [f"{address}{link}" for link in links]  # Adicionando o endereço base
})

# Exibindo o DataFrame
df


,Name,Link
0,I'm Still Here,https://letterboxd.com/film/im-still-here-2024/
1,Despicable Me 4,https://letterboxd.com/film/despicable-me-4/
2,Inside Out 2,https://letterboxd.com/film/inside-out-2-2024/
3,HAIKYU!! The Dumpster Battle,https://letterboxd.com/film/haikyu-the-dumpste...
4,Spider-Man: Across the Spider-Verse,https://letterboxd.com/film/spider-man-across-...
...,...,...
471,Modern Times,https://letterboxd.com/film/modern-times/
472,City Lights,https://letterboxd.com/film/city-lights/
473,Sunrise: A Song of Two Humans,https://letterboxd.com/film/sunrise-a-song-of-...
474,Leone XIII,https://letterboxd.com/film/leone-xiii/


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# Definindo os cabeçalhos para a requisição
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

BASE_URL = "https://letterboxd.com"

# Função para buscar o número total de páginas no perfil do usuário
def get_total_pages(user):
    response = requests.get(f"{BASE_URL}/{user}/films/page/1", headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Pega o número da última página na paginação
    pagination = soup.findAll('li', {'class': 'paginate-page'})
    if pagination:
        last_page = int(re.search(r'>(\d+)<', str(pagination[-1])).group(1))
    else:
        last_page = 1  # Caso não exista paginação, há apenas uma página.
    return last_page

# Função para extrair os filmes de uma página
def extract_films_from_page(user, page):
    response = requests.get(f"{BASE_URL}/{user}/films/page/{page}", headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')

    film_names = []
    film_links = []

    # Procurando todos os elementos de filmes
    for film_item in soup.findAll('li', {"class": "poster-container"}):
        name_match = re.search(r'img alt="(.*?)"', str(film_item))
        link_match = re.search(r'data-target-link="(.*?)"', str(film_item))
        
        if name_match and link_match:
            film_names.append(name_match.group(1))
            film_links.append(f"{BASE_URL}{link_match.group(1)}")  # Adiciona o endereço base ao link
    return film_names, film_links

# Função principal para coletar todos os filmes
def get_all_films(user):
    total_pages = get_total_pages(user)
    all_names = []
    all_links = []

    # Itera sobre todas as páginas do usuário
    for page in tqdm(range(1, total_pages + 1), desc="Fetching films"):
        film_names, film_links = extract_films_from_page(user, page)
        all_names.extend(film_names)
        all_links.extend(film_links)
    
    return pd.DataFrame({
        "Name": all_names,
        "Link": all_links
    })

# Execução principal
if __name__ == "__main__":
    user_profile = "kizardas"  # Substitua pelo nome de usuário desejado
    films_df = get_all_films(user_profile)
    
    # Exibindo o resultado
    print(films_df)
    # Salvar os resultados em um arquivo CSV, se necessário
    # films_df.to_csv(f"{user_profile}_data.csv", index=False)


Fetching films: 100%|██████████| 7/7 [00:06<00:00,  1.03it/s]

                             Name  \
0                 Despicable Me 4   
1                    Inside Out 2   
2              The Garfield Movie   
3    Aquaman and the Lost Kingdom   
4           The Boy and the Heron   
..                            ...   
499                Drunken Master   
500                    Cinderella   
501                  Modern Times   
502                  Sherlock Jr.   
503                        Omegle   

                                                  Link  
0         https://letterboxd.com/film/despicable-me-4/  
1       https://letterboxd.com/film/inside-out-2-2024/  
2      https://letterboxd.com/film/the-garfield-movie/  
3    https://letterboxd.com/film/aquaman-and-the-lo...  
4    https://letterboxd.com/film/the-boy-and-the-he...  
..                                                 ...  
499        https://letterboxd.com/film/drunken-master/  
500            https://letterboxd.com/film/cinderella/  
501          https://letterboxd.com/film/m

In [7]:
dta = films_df

pttrn = [r'[0-9][.][0-9][0-9]', r'>[A-Za-z]+<', r'[0-9]+', r'>.*?<']

data = dict()

for dt in tqdm(dta.values):
  r = requests.get(dt[1], headers=HEADERS)
  site = BeautifulSoup(r.text,'html.parser')
  
  if 'Not Found' in str(site.title):
    continue

  try:
    meta_string = str(site.find_all('meta',{'name':'twitter:data2'})[0])
    avr_rating = re.findall(pttrn[0],meta_string)[0]
  except IndexError:
      avr_rating=NaN
  try:
    meta_string = str(site.find_all('meta',{'name':'twitter:data2'})[0])
    avr_rating = re.findall(pttrn[0],meta_string)[0]
  except IndexError:
      avr_rating=NaN

  try:
    raw_duration = [str(x) for x in site.find_all('p', {'class':'text-link text-footer'})][0]
    duration = re.findall(pttrn[2], raw_duration)[0]
  except:
    duration = NaN
  try:
    raw_genre = site.find_all('div', {'class':'text-sluglist capitalize'})[0]
    raw_str = ''.join([str(x) for x in raw_genre.find_all('a')])
    genre = [x[1:-1] for x in re.findall(pttrn[3], raw_str)] 
  except:
      genre=[]
  
  try:
    language = str(site.select('a[href^="/films/language"]')[0])
    lang = re.findall(pttrn[3], language)[0][1:-1]

  except:
    lang=NaN


  data[f'{dt[0]}'] = [avr_rating,duration,genre,lang]

print('Search completed!')

year = pd.read_csv(r'letterboxd\watched.csv')
rating = pd.read_csv(r'letterboxd\ratings.csv')

raw = pd.DataFrame(data).T
raw = raw.reset_index()
raw.columns = ['Name','Avr_Rating','Duration','Genre','Language']

clean = pd.merge(raw,rating, on='Name', how='outer')

 88%|████████▊ | 446/504 [07:20<00:57,  1.01it/s]


NameError: name 'NaN' is not defined

In [6]:
site


<!DOCTYPE html>

<html class="no-mobile no-js" id="html" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="One by one, a flock of small birds perches on a telephone wire. Sitting close together has problems enough, and then comes along a large dopey bird that tries to join them. The birds of a feather can't help but make fun of him - and their clique mentality proves embarrassing in the end." name="description"/>
<meta content="video.movie" property="og:type"/>
<meta content="https://letterboxd.com/film/for-the-birds/" property="og:url"/>
<meta content="For the Birds (2000)" property="og:title"/>
<meta content="One by one, a flock of small birds perches on a telephone wire. Sitting close together has problems enough, and then comes along a large dopey bird that tries to join them. The birds of a feather can't help but make fun of him - and their clique mentality proves e